In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import SimpleRNN
from keras.layers import Dense,Flatten

Using TensorFlow backend.


In [2]:
data = pd.read_csv('GoogleStocks.csv')
data.head()

,date,close,volume,open,high,low
0,15:07,1169.41,1037784.0,1185.50,1187.5590,1159.370
1,2019/03/26,1184.62,1894639.0,1198.53,1202.8300,1176.720
2,2019/03/25,1193.00,1493841.0,1196.93,1206.3975,1187.040
3,2019/03/22,1205.50,1668910.0,1226.32,1230.0000,1202.825
4,2019/03/21,1231.54,1195899.0,1216.00,1231.7900,1213.150


In [3]:
low_arr = data['low'].values
high_arr = data['high'].values
avg_arr = (low_arr + high_arr)/2
data['avg'] = avg_arr
data.head()

,date,close,volume,open,high,low,avg
0,15:07,1169.41,1037784.0,1185.50,1187.5590,1159.370,1173.46450
1,2019/03/26,1184.62,1894639.0,1198.53,1202.8300,1176.720,1189.77500
2,2019/03/25,1193.00,1493841.0,1196.93,1206.3975,1187.040,1196.71875
3,2019/03/22,1205.50,1668910.0,1226.32,1230.0000,1202.825,1216.41250
4,2019/03/21,1231.54,1195899.0,1216.00,1231.7900,1213.150,1222.47000


In [4]:
data_train, data_test, out_train, out_test = train_test_split(
    data[['volume','avg']],
    data[['open']],
    test_size=0.2,
    random_state=0)
data_train.head()

,volume,avg
354,2027218.0,979.9000
252,2593808.0,1032.0150
97,1829295.0,1068.7925
522,2258695.0,824.3700
466,1596058.0,925.9600


In [5]:
out_train_arr = out_train[['open']].values
out_test_arr = out_test[['open']].values
train_ts = data_train[['volume','avg']].values
test_ts = data_test[['volume','avg']].values
sc = MinMaxScaler(feature_range=(0,1))
sc.fit(train_ts)
train_ts_scaled = sc.transform(train_ts)
test_ts_scaled = sc.transform(test_ts)
scy = MinMaxScaler(feature_range=(0,1))
scy.fit(out_train_arr)
out_train_scaled = scy.transform(out_train_arr)
out_test_scaled = scy.transform(out_test_arr)

In [6]:
H = [2,3]
C = [30,50,80]
T = [20,50,75]
n = 1
for h in H:
    for c in C:
        e = 50
        for t in T:
            print
            print '::::::::::::MODEL '+str(n)+' ::::::::::::::::'
            print 'Number of Hidden Layers : '+str(h)
            print 'Number of cells in Hidden Layers : '+str(c)
            print 'Number of Time Steps : '+str(t)
            print 'Number of Epochs : '+str(e)
            print
            X_train = []
            y_train = []
            X_test = []
            y_test = []
            for i in range(t, train_ts_scaled.shape[0]):
                X_train.append(train_ts_scaled[i-t:i, :])
                y_train.append(out_train_scaled[i, 0])
            X_train, y_train = np.array(X_train), np.array(y_train)
            for i in range(t, test_ts_scaled.shape[0]):
                X_test.append(test_ts_scaled[i-t:i, :])
                y_test.append(out_test_scaled[i, 0])
            X_test, y_test = np.array(X_test), np.array(y_test)
            model = Sequential()
            for m in range(0,h):
                model.add(SimpleRNN(units=c,return_sequences=True))
            model.add(Flatten())
            model.add(Dense(units = 1))
            model.compile(optimizer = 'adam', loss = 'mean_squared_error')
            model.fit(X_train, y_train, epochs = e, batch_size = 32)
            err = model.evaluate(X_test, y_test)
            print
            print 'Model '+str(n)+' error : '+str(err)
            print
            print '================================================================='
            e = e/2
            n += 1


::::::::::::MODEL 1 ::::::::::::::::
Number of Hidden Layers : 2
Number of cells in Hidden Layers : 30
Number of Time Steps : 20
Number of Epochs : 50

Epoch 1/50
584/584 [==============================] - 0s 738us/step - loss: 0.1853
Epoch 2/50
584/584 [==============================] - 0s 174us/step - loss: 0.0908
Epoch 3/50
584/584 [==============================] - 0s 191us/step - loss: 0.0818
Epoch 4/50
584/584 [==============================] - 0s 178us/step - loss: 0.0770
Epoch 5/50
584/584 [==============================] - 0s 191us/step - loss: 0.0747
Epoch 6/50
584/584 [==============================] - 0s 212us/step - loss: 0.0738
Epoch 7/50
584/584 [==============================] - 0s 180us/step - loss: 0.0753
Epoch 8/50
584/584 [==============================] - 0s 191us/step - loss: 0.0733
Epoch 9/50
584/584 [==============================] - 0s 178us/step - loss: 0.0716
Epoch 10/50
584/584 [==============================] - 0s 209us/step - loss: 0.0702
Epoch 11/50
584/

Epoch 1/50
584/584 [==============================] - 1s 902us/step - loss: 0.1610
Epoch 2/50
584/584 [==============================] - 0s 199us/step - loss: 0.1081
Epoch 3/50
584/584 [==============================] - 0s 201us/step - loss: 0.0767
Epoch 4/50
584/584 [==============================] - 0s 192us/step - loss: 0.0733
Epoch 5/50
584/584 [==============================] - 0s 228us/step - loss: 0.0800
Epoch 6/50
584/584 [==============================] - 0s 208us/step - loss: 0.0760
Epoch 7/50
584/584 [==============================] - 0s 233us/step - loss: 0.0713
Epoch 8/50
584/584 [==============================] - 0s 212us/step - loss: 0.0707
Epoch 9/50
584/584 [==============================] - 0s 193us/step - loss: 0.0735
Epoch 10/50
584/584 [==============================] - 0s 215us/step - loss: 0.0724
Epoch 11/50
584/584 [==============================] - 0s 203us/step - loss: 0.0681
Epoch 12/50
584/584 [==============================] - 0s 198us/step - loss: 0.0682
E

Epoch 1/50
584/584 [==============================] - 1s 1ms/step - loss: 0.1634
Epoch 2/50
584/584 [==============================] - 0s 224us/step - loss: 0.1047
Epoch 3/50
584/584 [==============================] - 0s 235us/step - loss: 0.0798
Epoch 4/50
584/584 [==============================] - 0s 237us/step - loss: 0.0835
Epoch 5/50
584/584 [==============================] - 0s 253us/step - loss: 0.0892
Epoch 6/50
584/584 [==============================] - 0s 259us/step - loss: 0.0804
Epoch 7/50
584/584 [==============================] - 0s 265us/step - loss: 0.0839
Epoch 8/50
584/584 [==============================] - 0s 285us/step - loss: 0.0697
Epoch 9/50
584/584 [==============================] - 0s 263us/step - loss: 0.0745
Epoch 10/50
584/584 [==============================] - 0s 259us/step - loss: 0.0765
Epoch 11/50
584/584 [==============================] - 0s 253us/step - loss: 0.0814
Epoch 12/50
584/584 [==============================] - 0s 253us/step - loss: 0.0758
Epo

Epoch 1/50
584/584 [==============================] - 1s 2ms/step - loss: 0.3051
Epoch 2/50
584/584 [==============================] - 0s 244us/step - loss: 0.1056
Epoch 3/50
584/584 [==============================] - 0s 238us/step - loss: 0.0779
Epoch 4/50
584/584 [==============================] - 0s 350us/step - loss: 0.0765
Epoch 5/50
584/584 [==============================] - 0s 244us/step - loss: 0.0727
Epoch 6/50
584/584 [==============================] - 0s 226us/step - loss: 0.0738
Epoch 7/50
584/584 [==============================] - 0s 225us/step - loss: 0.0760
Epoch 8/50
584/584 [==============================] - 0s 238us/step - loss: 0.0703
Epoch 9/50
584/584 [==============================] - 0s 256us/step - loss: 0.0780
Epoch 10/50
584/584 [==============================] - 0s 319us/step - loss: 0.0716
Epoch 11/50
584/584 [==============================] - 0s 350us/step - loss: 0.0766
Epoch 12/50
584/584 [==============================] - 0s 309us/step - loss: 0.0718
Epo

Epoch 1/50
584/584 [==============================] - 2s 3ms/step - loss: 0.2113
Epoch 2/50
584/584 [==============================] - 0s 293us/step - loss: 0.0835
Epoch 3/50
584/584 [==============================] - 0s 378us/step - loss: 0.0805
Epoch 4/50
584/584 [==============================] - 0s 277us/step - loss: 0.0768
Epoch 5/50
584/584 [==============================] - 0s 287us/step - loss: 0.0761
Epoch 6/50
584/584 [==============================] - 0s 309us/step - loss: 0.0749
Epoch 7/50
584/584 [==============================] - 0s 287us/step - loss: 0.0868
Epoch 8/50
584/584 [==============================] - 0s 307us/step - loss: 0.0765
Epoch 9/50
584/584 [==============================] - 0s 353us/step - loss: 0.0802
Epoch 10/50
584/584 [==============================] - 0s 303us/step - loss: 0.0895
Epoch 11/50
584/584 [==============================] - 0s 332us/step - loss: 0.0739
Epoch 12/50
584/584 [==============================] - 0s 371us/step - loss: 0.0748
Epo

584/584 [==============================] - 2s 3ms/step - loss: 0.4594
Epoch 2/50
584/584 [==============================] - 0s 454us/step - loss: 0.0921
Epoch 3/50
584/584 [==============================] - 0s 632us/step - loss: 0.0848
Epoch 4/50
584/584 [==============================] - 0s 574us/step - loss: 0.0760
Epoch 5/50
584/584 [==============================] - 0s 504us/step - loss: 0.0818
Epoch 6/50
584/584 [==============================] - 0s 377us/step - loss: 0.0695
Epoch 7/50
584/584 [==============================] - 0s 463us/step - loss: 0.0718
Epoch 8/50
584/584 [==============================] - 0s 490us/step - loss: 0.0670
Epoch 9/50
584/584 [==============================] - 0s 492us/step - loss: 0.0734
Epoch 10/50
584/584 [==============================] - 0s 469us/step - loss: 0.0706
Epoch 11/50
584/584 [==============================] - 0s 395us/step - loss: 0.0717
Epoch 12/50
584/584 [==============================] - 0s 432us/step - loss: 0.0797
Epoch 13/50
58